In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import logging
import gzip
import gensim 
import re
import spacy
import math
from bs4 import BeautifulSoup
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(torch.cuda.is_available())
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

True


'cuda:0'

# Preprocessing & Loading data

### Covid dataset

In [ ]:
# Load dataset for clothing reviews
covid_train = pd.read_csv("/content/drive/MyDrive/data /Corona_NLP_test.csv", encoding="latin1")
covid_train = covid_train.dropna()
covid_test = pd.read_csv("/content/drive/MyDrive/data /Corona_NLP_test.csv")
covid_test = covid_test.dropna()

covid_trian_inds = list(range(0, covid_train.shape[0]))

# cat both datasets for preprocessing
frames = [covid_train, covid_test]
covid = pd.concat(frames)
print(covid_train.head())
print(covid_test.shape)
print(type(covid))

   UserName  ...           Sentiment
0         1  ...  Extremely Negative
1         2  ...            Positive
3         4  ...            Negative
4         5  ...             Neutral
5         6  ...             Neutral

[5 rows x 6 columns]
(2964, 6)
<class 'pandas.core.frame.DataFrame'>


In [ ]:
covid.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
5,6,44958,Los Angeles,03-03-2020,Do you remember the last time you paid $2.99 a...,Neutral


In [ ]:
# preprocess training and testing set

covid = covid[['OriginalTweet', 'Sentiment']]
covid.columns = ['tweet', 'sentiment']
sentiment_dict = {'negative': 0, 'extremely negative':0, 'neutral':1, 'positive':2, 'extremely positive':2}
error_line = []

def encode_sentiment(x):
    sent_encoded = sentiment_dict[x.lower()]
    return sent_encoded

covid['sentiment_encoded'] = covid['sentiment'].apply(lambda x: encode_sentiment(x))



In [ ]:
covid.head()

,tweet,sentiment,sentiment_encoded
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,0
1,When I couldn't find hand sanitizer at Fred Me...,Positive,2
3,#Panic buying hits #NewYork City as anxious sh...,Negative,0
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,1
5,Do you remember the last time you paid $2.99 a...,Neutral,1


In [ ]:
print(len(covid.sentiment_encoded))
print(covid.shape[0])

5928
5928


#### Tokenize each sentence

In [ ]:
# read in data
def clear_data(df):
    stopword = set(stopwords.words('english'))

    # tokenize and post process, remove unnecessary words appearing in tweets(like urls and usernames)
    tweet_words = []
    str_lis = []
    for t in df['tweet']:
        t = t.lower()
        tokens = tokenize(t)
        tmp = []
        for token in tokens:
            # if token in stopword:
            #     continue
            if not wordnet.synsets(token):
                continue
            # repalce usernames with @USER
            user = re.sub(r'@[\w\W]+', '@USER', token)
            # replace with URL
            url = re.sub(r'https[\w\W]+','URL',user)
            if url in stopword:
                continue
            tmp.append(url)
        tweet_words.append(tmp)
        st = ' '.join([str(item) for item in tmp])
        str_lis.append(st)
    df['tweet_words'] = tweet_words
    df['tweets'] = str_lis

    # remove duplicates
    df.drop_duplicates(subset=['tweets'], inplace=True)

    df.drop(columns='tweet_words', inplace=True)
    print(len(df))
    df.to_csv("covid.csv", index=False)
    return df

def tokenize(text):
    words = []
    # print(text.split())
    for token in text.split():
        # find wouldn't kind word
        words.extend(re.findall(r"\w+-\w+|https.+|\.+|\d+[\.,]\d+|[@#]\w+|[+-]\d+|(?:(?!n[’'])\w)+|\w?[’']\w+|[^\s\w]", token))
        # words.extend(re.findall(r"(?:(?!.')\w)+|\w?'\w+|[^\s\w]", token))
    return words

In [ ]:
covid = clear_data(covid)

3086


In [ ]:
covid.head()

,tweet,sentiment,sentiment_encoded,tweets
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,0,trending new encounter empty supermarket shelv...
1,When I couldn't find hand sanitizer at Fred Me...,Positive,2,find hand turned 2 pack check concerns driving...
3,#Panic buying hits #NewYork City as anxious sh...,Negative,0,buying hits city anxious shoppers stock food a...
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,1,one week buying baby milk powder next buying t...
5,Do you remember the last time you paid $2.99 a...,Neutral,1,remember last time paid gallon regular gas pri...


#### Encode and truncate sentence

In [ ]:
# tokenize the tweets into list
def encode_tweets(x):
    tweet_list = x.split()
    return tweet_list

covid['tweets_tokenized'] = covid['tweets'].apply(lambda x: encode_tweets(x))
covid.reset_index(drop=True, inplace=True)
# drop rows wth empty tokenized tweet
for i, row in covid.iterrows():
    if len(row.tweets_tokenized) < 2:
        covid.drop(i, inplace = True)

In [ ]:
covid.head(30)

,tweet,sentiment,sentiment_encoded,tweets,tweets_tokenized
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,0,trending new encounter empty supermarket shelv...,"[trending, new, encounter, empty, supermarket,..."
1,When I couldn't find hand sanitizer at Fred Me...,Positive,2,find hand turned 2 pack check concerns driving...,"[find, hand, turned, 2, pack, check, concerns,..."
2,#Panic buying hits #NewYork City as anxious sh...,Negative,0,buying hits city anxious shoppers stock food a...,"[buying, hits, city, anxious, shoppers, stock,..."
3,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,1,one week buying baby milk powder next buying t...,"[one, week, buying, baby, milk, powder, next, ..."
4,Do you remember the last time you paid $2.99 a...,Neutral,1,remember last time paid gallon regular gas pri...,"[remember, last, time, paid, gallon, regular, ..."
5,"@DrTedros ""We canÂt stop #COVID19 without pro...",Neutral,1,stop protecting prices surgical masks increase...,"[stop, protecting, prices, surgical, masks, in..."
6,Anyone been in a supermarket over the last few...,Extremely Positive,2,supermarket last days went normal shop last ni...,"[supermarket, last, days, went, normal, shop, ..."
7,Best quality couches at unbelievably low price...,Positive,2,best quality couches unbelievably low prices a...,"[best, quality, couches, unbelievably, low, pr..."
8,Beware of counterfeits trying to sell fake mas...,Extremely Negative,0,beware counterfeits trying sell fake masks che...,"[beware, counterfeits, trying, sell, fake, mas..."
9,Panic food buying in Germany due to #coronavir...,Extremely Negative,0,panic food buying germany due begun left behin...,"[panic, food, buying, germany, due, begun, lef..."


In [ ]:
# sanity check for length of tweet
len_list = [len(x) for x in covid.tweets_tokenized]
max(len_list)

34

In [ ]:
# get unique words in the corpus
all_words = []
for x in covid['tweets_tokenized']:
    all_words.extend(x)

word_set = list(set(all_words))
word_count = Counter(all_words)

# filter out words with low frequency
for word_list in covid.tweets_tokenized:
    new_list = []
    for word in word_list:
        if word_count[word] > 2:
            new_list.append(word)
    word_list = new_list

# update set of words after removing the ones with low frequency
new_word_list = []
for x in covid['tweets_tokenized']:
    new_word_list.extend(x)
word_set = list(set(new_word_list))

# map each unique words & unknown token in covid.encoded to an index
word2index = {}
word2index['<UNK>'] = 0
word2index['<PAD>'] = 1

for i, word in enumerate(word_set, 2):
    word2index[word] = i

# encode the original sequence
def encode(sent_list):
    result = []
    for x in sent_list:
        index = word2index[x]
        result.append(index)
    return result

covid['encoded_old'] = covid.tweets_tokenized.apply(lambda x: encode(x))


# get sequence average length
total_len = 0
for x in covid.encoded_old:
    total_len += len(x)

ave_len = math.floor(total_len/covid.shape[0])

# filter out long sequences --> encode all sequence to length = ave_len
# pad short sequence

def fix_tweet(x):
    size = min(len(x), ave_len)
    new_encoded = x[:size]
    len_before_pad = len(new_encoded)
    if size < ave_len:
        for j in range(0, ave_len - len(x)):
            new_encoded.append(0)
    return new_encoded

covid['encoded'] = covid.encoded_old.apply(lambda x: fix_tweet(x))
covid['tweet_length'] = covid.encoded_old.apply(lambda x: min(len(x), ave_len))

In [ ]:
max(covid.tweet_length)

print(word2index['exchange'])
covid.head()

4553


,tweet,sentiment,sentiment_encoded,tweets,tweets_tokenized,encoded_old,encoded,tweet_length
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,0,trending new encounter empty supermarket shelv...,"[trending, new, encounter, empty, supermarket,...","[3126, 3423, 6424, 1857, 790, 5008, 1469, 1538...","[3126, 3423, 6424, 1857, 790, 5008, 1469, 1538...",14
1,When I couldn't find hand sanitizer at Fred Me...,Positive,2,find hand turned 2 pack check concerns driving...,"[find, hand, turned, 2, pack, check, concerns,...","[6367, 2030, 4112, 6211, 1242, 253, 2849, 4985...","[6367, 2030, 4112, 6211, 1242, 253, 2849, 4985...",9
2,#Panic buying hits #NewYork City as anxious sh...,Negative,0,buying hits city anxious shoppers stock food a...,"[buying, hits, city, anxious, shoppers, stock,...","[6499, 5482, 3627, 6299, 435, 3588, 6658, 6305...","[6499, 5482, 3627, 6299, 435, 3588, 6658, 6305...",15
3,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,1,one week buying baby milk powder next buying t...,"[one, week, buying, baby, milk, powder, next, ...","[6660, 323, 6499, 5283, 968, 225, 2529, 6499, ...","[6660, 323, 6499, 5283, 968, 225, 2529, 6499, ...",10
4,Do you remember the last time you paid $2.99 a...,Neutral,1,remember last time paid gallon regular gas pri...,"[remember, last, time, paid, gallon, regular, ...","[6247, 5791, 254, 5818, 6153, 4217, 3031, 5969...","[6247, 5791, 254, 5818, 6153, 4217, 3031, 5969...",13


In [ ]:
# sanity check for encoding:
print(len(all_words))
print(ave_len)
lengths = [len(x) for x in covid.encoded]

tweet_length = covid.tweet_length
covid.head()


47853
15


,tweet,sentiment,sentiment_encoded,tweets,tweets_tokenized,encoded_old,encoded,tweet_length
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative,0,trending new encounter empty supermarket shelv...,"[trending, new, encounter, empty, supermarket,...","[3126, 3423, 6424, 1857, 790, 5008, 1469, 1538...","[3126, 3423, 6424, 1857, 790, 5008, 1469, 1538...",14
1,When I couldn't find hand sanitizer at Fred Me...,Positive,2,find hand turned 2 pack check concerns driving...,"[find, hand, turned, 2, pack, check, concerns,...","[6367, 2030, 4112, 6211, 1242, 253, 2849, 4985...","[6367, 2030, 4112, 6211, 1242, 253, 2849, 4985...",9
2,#Panic buying hits #NewYork City as anxious sh...,Negative,0,buying hits city anxious shoppers stock food a...,"[buying, hits, city, anxious, shoppers, stock,...","[6499, 5482, 3627, 6299, 435, 3588, 6658, 6305...","[6499, 5482, 3627, 6299, 435, 3588, 6658, 6305...",15
3,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral,1,one week buying baby milk powder next buying t...,"[one, week, buying, baby, milk, powder, next, ...","[6660, 323, 6499, 5283, 968, 225, 2529, 6499, ...","[6660, 323, 6499, 5283, 968, 225, 2529, 6499, ...",10
4,Do you remember the last time you paid $2.99 a...,Neutral,1,remember last time paid gallon regular gas pri...,"[remember, last, time, paid, gallon, regular, ...","[6247, 5791, 254, 5818, 6153, 4217, 3031, 5969...","[6247, 5791, 254, 5818, 6153, 4217, 3031, 5969...",13


In [ ]:
# Train test split from skearln
data_size = len(covid['encoded'])
assert data_size == len(covid['sentiment_encoded']) 
X, y = list(zip(list(covid['encoded']),(list(covid['tweet_length'])))), list(covid['sentiment_encoded'])
# X(data, length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [ ]:
X_train_new=[]
for tup in X_train:
  arr = np.asarray(tup)
  X_train_new.append(arr)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
X_test_new=[]
for tup in X_test:
  arr = np.asarray(tup)
  X_test_new.append(arr)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(np.array(self.X[idx][0])), self.y[idx], self.X[idx][1]

In [ ]:
train_ds = ReviewsDataset(X_train_new, y_train)
valid_ds = ReviewsDataset(X_test_new, y_test)

In [ ]:
type(X_train_new[0][1])

int

In [ ]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    epoch = 0
    best_val_acc = 0.0  
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        batch_count=0
        total = 0
        train_correct = 0.0
        epoch += 1
        print(f"At epoch {i}")
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            batch_count+=1
            optimizer.zero_grad()
            y_pred = model(x, l)
            pred = torch.argmax(y_pred, 1)
            train_correct += sum((pred == y))
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, precision, recall, f1  = validation_metrics(model, val_dl)
        print("train loss %.3f, train accuracy %.3f, val loss %.3f, val accuracy %.3f, precision %.3f, recall %.3f, F1 %.3f" 
              % (sum_loss/total, train_correct/total, val_loss, val_acc, precision, recall, f1))
        if val_acc > best_val_acc and i>=1:
            best_val_acc = val_acc

            #torch.save(model.state_dict(), NEW_PATH)
            print(f"\t=> Best model saved at {i}th epoch with valication accuracy of {val_acc}")
def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    y_total = []
    y_pred_total = []
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        y_total.extend(y.tolist())
        y_pred_total.extend(pred.tolist())
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        #sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]

    f1 = (f1_score(y_total, y_pred_total, average='weighted'))
    precision = (precision_score(y_total, y_pred_total, average='weighted'))
    recall =(recall_score(y_total, y_pred_total, average='weighted'))
   
    return sum_loss/total, correct/total,  precision, recall, f1


In [ ]:
batch_size = 5000
vocab_size = len(word2index)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [ ]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [ ]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)

In [ ]:
train_model(model_fixed, epochs=30, lr=0.01)

At epoch 0
train loss 1.584, train accuracy 0.367, val loss 1.512, val accuracy 0.419, precision 0.372, recall 0.419, F1 0.301
At epoch 1


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is 

train loss 1.508, train accuracy 0.439, val loss 1.430, val accuracy 0.425, precision 0.370, recall 0.425, F1 0.345
	=> Best model saved at 1th epoch with valication accuracy of 0.4253246784210205
At epoch 2
train loss 1.422, train accuracy 0.463, val loss 1.312, val accuracy 0.417, precision 0.351, recall 0.417, F1 0.381
At epoch 3


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train loss 1.302, train accuracy 0.472, val loss 1.171, val accuracy 0.430, precision 0.398, recall 0.430, F1 0.339
	=> Best model saved at 3th epoch with valication accuracy of 0.43019479513168335
At epoch 4
train loss 1.168, train accuracy 0.443, val loss 1.133, val accuracy 0.435, precision 0.454, recall 0.435, F1 0.308
	=> Best model saved at 4th epoch with valication accuracy of 0.4350649416446686
At epoch 5


...

train loss 0.964, train accuracy 0.490, val loss 1.009, val accuracy 0.437, precision 0.373, recall 0.437, F1 0.391
At epoch 14
train loss 0.950, train accuracy 0.537, val loss 1.012, val accuracy 0.419, precision 0.357, recall 0.419, F1 0.360
At epoch 15


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train loss 0.939, train accuracy 0.537, val loss 1.016, val accuracy 0.419, precision 0.357, recall 0.419, F1 0.355
At epoch 16
train loss 0.932, train accuracy 0.541, val loss 1.013, val accuracy 0.429, precision 0.368, recall 0.429, F1 0.373
At epoch 17


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train loss 0.910, train accuracy 0.555, val loss 1.005, val accuracy 0.440, precision 0.376, recall 0.440, F1 0.393
At epoch 18
train loss 0.880, train accuracy 0.569, val loss 1.003, val accuracy 0.464, precision 0.393, recall 0.464, F1 0.423
	=> Best model saved at 18th epoch with valication accuracy of 0.4642857015132904
At epoch 19
train loss 0.858, train accuracy 0.579, val loss 1.011, val accuracy 0.489, precision 0.489, recall 0.489, F1 0.488
	=> Best model saved at 19th epoch with valication accuracy of 0.4886363744735718
At epoch 20
train loss 0.834, train accuracy 0.625, val loss 1.026, val accuracy 0.472, precision 0.495, recall 0.472, F1 0.472
At epoch 21
train loss 0.812, train accuracy 0.611, val loss 1.035, val accuracy 0.463, precision 0.481, recall 0.463, F1 0.464
At epoch 22
train loss 0.793, train accuracy 0.628, val loss 1.029, val accuracy 0.489, precision 0.500, recall 0.489, F1 0.491
At epoch 23
train loss 0.754, train accuracy 0.656, val loss 1.025, val accuracy

In [ ]:
train_model(model_fixed, epochs=30, lr=0.01)

At epoch 0
train loss 0.563, train accuracy 0.766, val loss 1.282, val accuracy 0.416, precision 0.525, recall 0.416, F1 0.420
At epoch 1
train loss 0.760, train accuracy 0.674, val loss 1.097, val accuracy 0.510, precision 0.546, recall 0.510, F1 0.517
	=> Best model saved at 1th epoch with valication accuracy of 0.5097402334213257
At epoch 2
train loss 0.569, train accuracy 0.761, val loss 1.089, val accuracy 0.554, precision 0.566, recall 0.554, F1 0.542
	=> Best model saved at 2th epoch with valication accuracy of 0.5535714030265808
At epoch 3
train loss 0.540, train accuracy 0.771, val loss 1.124, val accuracy 0.552, precision 0.569, recall 0.552, F1 0.530
At epoch 4
train loss 0.575, train accuracy 0.765, val loss 1.115, val accuracy 0.560, precision 0.571, recall 0.560, F1 0.539
	=> Best model saved at 4th epoch with valication accuracy of 0.5600649118423462
At epoch 5
train loss 0.536, train accuracy 0.762, val loss 1.082, val accuracy 0.580, precision 0.584, recall 0.580, F1 0

In [ ]:
train_model(model_fixed, epochs=30, lr=0.01)

At epoch 0
train loss 0.138, train accuracy 0.959, val loss 1.468, val accuracy 0.563, precision 0.598, recall 0.563, F1 0.568
At epoch 1
train loss 0.251, train accuracy 0.914, val loss 1.499, val accuracy 0.610, precision 0.614, recall 0.610, F1 0.608
	=> Best model saved at 1th epoch with valication accuracy of 0.6103895902633667
At epoch 2
train loss 0.135, train accuracy 0.963, val loss 1.642, val accuracy 0.607, precision 0.617, recall 0.607, F1 0.595
At epoch 3
train loss 0.196, train accuracy 0.941, val loss 1.580, val accuracy 0.606, precision 0.613, recall 0.606, F1 0.596
At epoch 4
train loss 0.160, train accuracy 0.949, val loss 1.477, val accuracy 0.619, precision 0.619, recall 0.619, F1 0.616
	=> Best model saved at 4th epoch with valication accuracy of 0.6185064911842346
At epoch 5
train loss 0.095, train accuracy 0.973, val loss 1.439, val accuracy 0.597, precision 0.602, recall 0.597, F1 0.598
At epoch 6
train loss 0.110, train accuracy 0.968, val loss 1.443, val accur